**Feel free check our [Youtube Link](https://www.youtube.com/watch?v=8g9U342bfrU&feature=youtu.be)**

# Create and Save Model

## Initialization

In [1]:
import tensorflow as tf

BATCH_SIZE = 100
EPOCHS = 15
checkpoint_path = "training_1/cp.ckpt"
my_model_path = 'my_model.h5' # .h5 or.hdf5

**Neuron network**

In [2]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Conv2D(kernel_size=3, filters=12, use_bias=False, padding='same', input_shape=(28, 28, 1)),
  tf.keras.layers.BatchNormalization(center=True, scale=False),
  tf.keras.layers.Activation('relu'),
  
  tf.keras.layers.Conv2D(kernel_size=6, filters=24, use_bias=False, padding='same', strides=2),
  tf.keras.layers.BatchNormalization(center=True, scale=False),
  tf.keras.layers.Activation('relu'),
  
  tf.keras.layers.Conv2D(kernel_size=6, filters=32, use_bias=False, padding='same', strides=2),
  tf.keras.layers.BatchNormalization(center=True, scale=False),
  tf.keras.layers.Activation('relu'),
  ## ↑ convolution ↑ #### ↓ fully-connected layer ↓ ##
  tf.keras.layers.Flatten(),
  
  tf.keras.layers.Dense(200, use_bias=False),
  tf.keras.layers.BatchNormalization(center=True, scale=False),
  tf.keras.layers.Activation('relu'),
  
  tf.keras.layers.Dropout(0.3), # To solve overfitting
  tf.keras.layers.Dense(10, activation='softmax')
])

print(model.output_shape)
model.summary()

model.compile(optimizer='adam',  # or 'sgd', tf.keras.optimizers.Adam() or tf.train.AdamOptimizer()
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

(None, 10)
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 28, 28, 12)        108       
_________________________________________________________________
batch_normalization (BatchNo (None, 28, 28, 12)        36        
_________________________________________________________________
activation (Activation)      (None, 28, 28, 12)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 14, 14, 24)        10368     
_________________________________________________________________
batch_normalization_1 (Batch (None, 14, 14, 24)        72        
_________________________________________________________________
activation_1 (Activation)    (None, 14, 14, 24)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 7, 7, 32)

**Import datasets**

In [3]:
mnist = tf.keras.datasets.mnist
(training_data, training_labels),(validation_data, validation_labels) = mnist.load_data()

image_size=28

training_data = training_data.reshape(training_data.shape[0], image_size, image_size, 1).astype('float32') # .astype('float32')
validation_data = validation_data.reshape(validation_data.shape[0], image_size, image_size, 1).astype('float32')             # .astype('float32')

training_data = tf.keras.utils.normalize(training_data, axis=1) # training_data, validation_data = training_data/255, validation_data/255
validation_data = tf.keras.utils.normalize(validation_data, axis=1)

print(training_data.dtype)
print(training_data.shape)
print(validation_data.dtype)
print(validation_data.shape)

float32
(60000, 28, 28, 1)
float32
(10000, 28, 28, 1)


## Training & evaluating

In [4]:
import math

# steps_per_epoch & validation_steps
compute_steps_per_epoch = lambda x: int(math.ceil(1. * x / BATCH_SIZE))    # BATCH_SIZE = 100
StepsPerEpoch = compute_steps_per_epoch(training_data.shape[0])            # training_data.shape[0] = 60000 (TRAINING SIZE)
valSteps = compute_steps_per_epoch(validation_data.shape[0])                     # validation_data.shape[0] = 10000 (VALIDATION SIZE)
print("Steps per epoch: ", StepsPerEpoch)

# callbacks 1 - define the checkpoint
model_checkpoint = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path,
    save_weights_only=True,
    monitor='val_loss',
    mode='min',
    save_best_only=True)

# callbacks 2 - define the lr decay
def lr_decay(epoch):
  return 0.01 * math.pow(0.6, epoch)
lr_decay_callback = tf.keras.callbacks.LearningRateScheduler(lr_decay, verbose=True)

# model fit
model.fit(training_data, training_labels, steps_per_epoch=StepsPerEpoch, epochs=EPOCHS, shuffle=True,\
          validation_data=(validation_data, validation_labels), validation_steps=valSteps, callbacks=[model_checkpoint, lr_decay_callback])

Steps per epoch:  600

Epoch 00001: LearningRateScheduler reducing learning rate to 0.01.
Epoch 1/15
600/600 [==============================] - 80s 134ms/step - loss: 0.1260 - accuracy: 0.9617 - val_loss: 0.0641 - val_accuracy: 0.9796 - lr: 0.0100

Epoch 00002: LearningRateScheduler reducing learning rate to 0.006.
Epoch 2/15
600/600 [==============================] - 80s 133ms/step - loss: 0.0442 - accuracy: 0.9860 - val_loss: 0.0324 - val_accuracy: 0.9895 - lr: 0.0060

Epoch 00003: LearningRateScheduler reducing learning rate to 0.0036.
Epoch 3/15
600/600 [==============================] - 80s 133ms/step - loss: 0.0242 - accuracy: 0.9927 - val_loss: 0.0339 - val_accuracy: 0.9896 - lr: 0.0036

Epoch 00004: LearningRateScheduler reducing learning rate to 0.0021599999999999996.
Epoch 4/15
600/600 [==============================] - 80s 133ms/step - loss: 0.0152 - accuracy: 0.9954 - val_loss: 0.0216 - val_accuracy: 0.9931 - lr: 0.0022

Epoch 00005: LearningRateScheduler reducing learning 

## Save the model

In [8]:
model.save(my_model_path)

# Reload Model

## initialization

**load model**

PS. Remember to upload "my_model.h5" file before start runtime.

In [9]:
import tensorflow as tf

BATCH_SIZE = 100
EPOCHS = 10
checkpoint_path = "training_1/cp.ckpt"
new_model_path = 'my_model.h5'
new_Model = tf.keras.models.load_model(
    new_model_path)

**load datasets**

In [10]:
mnist = tf.keras.datasets.mnist
(training_data, training_labels),(validation_data, validation_labels) = mnist.load_data()

image_size=28

training_data = training_data.reshape(training_data.shape[0], image_size, image_size, 1).astype('float32') # .astype('float32')
validation_data = validation_data.reshape(validation_data.shape[0], image_size, image_size, 1).astype('float32')             # .astype('float32')

training_data = tf.keras.utils.normalize(training_data, axis=1) # training_data, validation_data = training_data/255, validation_data/255
validation_data = tf.keras.utils.normalize(validation_data, axis=1)

## Evaluating

In [11]:
val_loss, val_acc = new_Model.evaluate(validation_data, validation_labels)

313/313 [==============================] - 4s 12ms/step - loss: 0.0221 - accuracy: 0.9938


In [ ]:
# Pick one picture to validate
print("Label:" + str(validation_labels[0]))
predictions = new_Model.predict(validation_data)
import numpy as np
print("Prediction" + str(np.argmax(predictions[0])))

## Continue training

In [ ]:
# Training, evaluating

import math

# steps_per_epoch & validation_steps
compute_steps_per_epoch = lambda x: int(math.ceil(1. * x / BATCH_SIZE))    # BATCH_SIZE = 100
StepsPerEpoch = compute_steps_per_epoch(training_data.shape[0])            # training_data.shape[0] = 60000 (TRAINING SIZE)
valSteps = compute_steps_per_epoch(validation_data.shape[0])                     # validation_data.shape[0] = 10000 (VALIDATION SIZE)
print("Steps per epoch: ", StepsPerEpoch)

# callbacks 1 - define the checkpoint
model_checkpoint = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path,
    save_weights_only=True,
    monitor='val_loss',
    mode='min',
    save_best_only=True)

# callbacks 2 - define the lr decay
def lr_decay(epoch):
  return 0.01 * math.pow(0.6, epoch)
lr_decay_callback = tf.keras.callbacks.LearningRateScheduler(lr_decay, verbose=True)

# new model fit
new_Model.fit(training_data, training_labels, steps_per_epoch=StepsPerEpoch, epochs=EPOCHS, shuffle=True,\
          validation_data=(validation_data, validation_labels), validation_steps=valSteps, callbacks=[model_checkpoint, lr_decay_callback])

**save new model**

In [ ]:
new_Model.save(new_model_path)

# Group 5

authors: 
*   **Hao-Bang Lu** C24031211
*   **Montenegro Aguilar Mauricio Adrian** F04077150